In [1]:
%matplotlib inline
import importlib
import numpy as np
import pandas as pd
import json
import utils

In [2]:
importlib.reload(utils)

<module 'utils' from '/home/fei/Documents/projects/lyrics/utils.py'>

In [3]:
dat = pd.read_csv('eng_songdata.csv', usecols=['text']).drop_duplicates()

In [4]:
dat.head()

,text
0,"Look at her face, it's a wonderful face \nAnd..."
1,"Take it easy with me, please \nTouch me gentl..."
2,I'll never know why I had to go \nWhy I had t...
3,Making somebody happy is a question of give an...
4,Making somebody happy is a question of give an...


In [5]:
glove = pd.read_table('glove.6B.50d.txt', sep=' ', header=None, quoting=3, na_filter=False, index_col=0)

In [6]:
# glove_words = set(glove.index.tolist())

In [7]:
dat.loc[:, 'text'] += '(end)'

In [8]:
cleaned = utils.data_cleaning(dat.copy())

In [9]:
cleaned.head()

0    [look, at, her, face, ,, it, 's, a, wonderful,...
1    [take, it, easy, with, me, ,, please, \n, touc...
2    [i, 'll, never, know, why, i, had, to, go, \n,...
3    [making, somebody, happy, is, a, question, of,...
4    [making, somebody, happy, is, a, question, of,...
Name: text_list, dtype: object

In [11]:
# try to remove non-english songs
# n_eng = cleaned.apply(lambda x: len(set(x) & glove_words))

In [12]:
#dat.loc[n_eng > 20].to_csv('eng_songdata.csv', index=False)

In [13]:
cleaned_sets = cleaned.apply(set)

In [15]:
vocab_size, unique_tokens = utils.get_unique_tokens(cleaned_sets)

In [16]:
vocab_size

81452

In [17]:
unique_tokens[:5]

['\n', '!', '"', "'", "'cause"]

In [30]:
utils.save_unique_tokens(unique_tokens)

In [32]:
word2ind, ind2word = utils.get_index_word_map(unique_tokens)

In [10]:
word2ind, ind2word = utils.load_index_word_map()

In [34]:
utils.save_index_word_map(word2ind, ind2word)

In [36]:
emb = utils.create_emb(ind2word, glove)

In [38]:
utils.save_embedding(emb)

In [11]:
tokenised = utils.tokenise_cleaned_data(cleaned, word2ind)

In [12]:
tokenised.head()

0    [41434, 3741, 31893, 23851, 19, 36007, 12, 36,...
1    [70486, 36007, 21300, 79589, 44029, 19, 54233,...
2    [33764, 7, 48341, 38656, 79162, 33764, 30328, ...
3    [42765, 66351, 30916, 35884, 36, 56819, 49792,...
4    [42765, 66351, 30916, 35884, 36, 56819, 49792,...
Name: text_list, dtype: object

In [13]:
%%time
samples = utils.make_training_samples(tokenised, input_length=100)

CPU times: user 1h 5min 45s, sys: 9min 56s, total: 1h 15min 41s
Wall time: 1h 15min 35s


In [14]:
samples.shape

(11373448, 101)

In [15]:
samples.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,41434,3741,31893,23851,19,36007,12,36,79788,23851,...,15209,33764,19682,33,0,0,2348,78828,78354,28755
1,3741,31893,23851,19,36007,12,36,79788,23851,0,...,33764,19682,33,0,0,2348,78828,78354,28755,25987
2,31893,23851,19,36007,12,36,79788,23851,0,2348,...,19682,33,0,0,2348,78828,78354,28755,25987,36
3,23851,19,36007,12,36,79788,23851,0,2348,36007,...,33,0,0,2348,78828,78354,28755,25987,36,77867
4,19,36007,12,36,79788,23851,0,2348,36007,44070,...,0,0,2348,78828,78354,28755,25987,36,77867,34505


In [16]:
utils.save_training_samples(samples)